---
title: "Reevaluating Automated Wildlife Species Detection: A Reproducibility Study on Public Image Datasets"
subtitle: "Testing Google’s Pre-trained Inception-ResNet-v2 Computer Vision Model on a Public Animal Image Dataset"
date: 2025-09-03
keywords: ["machine learning", "reproducibility", "animal species classification", "computer vision", "neural networks", "cnn", "resnet", "tensorflow", "wildlife monitoring"]
exports: 
  - format: pdf
    template: arxiv_nips
---

+++ {"part": "abstract"}
This experiment attempts to reproduce the results of the paper [Automated detection of European wild mammal species in camera trap images with an existing and pre-trained computer vision model](doi:10.1007/s10344-020-01404-y), 
which tests the pretrained Google Inception-ResNet-v2 model for predicting animal species.
We describe the required software, image loading processes, model outputs. Furthermore we calculate prediction global and per-class prediction accuracies and compare them to the metrics from the original paper.
+++

# Runtime Environment

The paper does not supply great insight into the runtime environment. This is likely due to the standardized way the model can be used through the TensorFlow library. To maximize the readability and reproducibility of the experiment, a minimal setup was chosen, defining all necessary code, data, and requirements in one GitHub project.
State-of-the-art python packages are chosen, installed, and imported. The exact versions are shown in {numref}`table-requirements`. The Jupyter notebook is run locally on a Thinkpad T14 with an AMD Ryzen 5 PRO 5650U processor, 16 GB memory and Linux Mint 22.1 installed.

In [1]:
!python3.12 -m pip install -r requirements.txt

In [2]:
# image & data processing
from pathlib import Path
from PIL import Image
import numpy as np
import pandas as pd
import tabulate
# neural network
import tensorflow as tf
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.applications.inception_resnet_v2 import decode_predictions
# evaluation
from sklearn.metrics import accuracy_score, confusion_matrix
# plot
import matplotlib.pyplot as plt
import seaborn as sns

2025-09-09 14:01:11.333533: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-09 14:01:11.338439: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-09-09 14:01:11.349040: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757419271.363482   98279 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757419271.367528   98279 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757419271.383581   98279 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

# Model
Once the Python runtime is set up and the packages are imported, the neural network can be instantiated and directly fit to the ImageNet dataset. This eliminates all model design and training work.

In [3]:
model = InceptionResNetV2(weights="imagenet")

2025-09-09 14:01:13.763255: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


# Data
For the experiment, 90 common animals are used. They are sourced from Google Images and provided in a labeled format in @banerjee2024animal. The Kaggle dataset is rather large. To mimic the original experiment setup, only 10 samples are used for each species, resulting in a total test sample size of 900 images.

In [4]:
DATA_PATH = Path("data/kaggle-90-different-animals")
model_input_size = model.input_shape[1:3] # the required image dimensions (299, 299)

## Data Preprocessing
The images are loaded with three color channels (RGB), resized to 299 by 299 pixels and converted into an 1-dimensional vector. The color intensities are scaled to be floating point numbers from 0 to 1. This is the minimal preprocessing required to fit the required input size of the neural network.

```{image} data/kaggle-90-different-animals/wombat/2a6c3fd292.jpg
:align: center
:alt: Example input image
```

In [5]:
def load_normalized_image(path, target_size=model_input_size):
    image = Image.open(path).convert("RGB")
    image = image.resize(target_size)
    return np.array(image) / 255.0

In [6]:
wildlife_image_paths = []

animal_species = sorted([d.name for d in DATA_PATH.iterdir() if d.is_dir()])

for species_name in animal_species:
    animal_image_folder = DATA_PATH / species_name # every species has its image folder
    for image_path in animal_image_folder.glob("*.jpg"):
        image_array = load_normalized_image(image_path)
        wildlife_image_paths.append(image_path)

The testing data is constructed by stacking the normalized image vectors and using the folder names as the label.

In [7]:
animal_images = [load_normalized_image(p) for p in wildlife_image_paths]
animal_species = [p.parent.name for p in wildlife_image_paths]

X_test = np.stack(animal_images, axis=0)
y_true = animal_species

# Test
The model yields a probability for each of the 1000 classes. The classes represent 1000 different classes taken from the ImageNet database. For this experiment, we use the output from the top neuron of the final softmax layer and compare its label to the true label.

In [8]:
y_pred = model.predict(X_test)
y_pred = [pred[0][1] for pred in decode_predictions(y_pred, top=1)]

2025-09-09 14:01:36.951283: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 965530800 exceeds 10% of free system memory.
2025-09-09 14:01:40.651288: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.
2025-09-09 14:01:40.835258: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123887616 exceeds 10% of free system memory.


 1/29 ━━━━━━━━━━━━━━━━━━━━ 2:55 6s/step

2025-09-09 14:01:44.325908: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 177020928 exceeds 10% of free system memory.
2025-09-09 14:01:44.536403: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 123887616 exceeds 10% of free system memory.


29/29 ━━━━━━━━━━━━━━━━━━━━ 109s 4s/step


When looking at the results, it becomes apparent that the model yields usable results. Almost all inference outputs are animal species somehow related to the one present in the image. This shows that the InceptionResNetV2 is generalizable to some extent.

In [9]:
species_recognition_result = pd.DataFrame({
    "y_true": y_true,
    "y_pred": y_pred
})
species_recognition_result

,y_true,y_pred
0,antelope,gazelle
1,antelope,impala
2,antelope,impala
3,antelope,gazelle
4,antelope,gazelle
...,...,...
895,zebra,zebra
896,zebra,zebra
897,zebra,zebra
898,zebra,zebra


## Label Mapping
The main issue with this experiment is the set of classes known to the model, which do not match the dataset used for testing. This is not specific to this dataset, but it is very likely to happen in any kind of realistic setup. We manually define a mapping table to relate the model output label to the labels from the dataset. 

This mapping is done manually, as a best-effort approach following the Linnean system of taxonomy, and we acknowledge some shortcomings of it:

- Some semantic information is lost as species are sometimes mapped to their families. (e.g., all bear species are mapped to bear)
- The dataset contains images of species that are not directly related to any class from the model. (e.g., all bats, deers)


 View {numref}`table-imagenet-label-mapping` for all mappings.

In [36]:
imagenet_to_kaggle = {
    "gazelle": "antelope",
    "impala": "antelope",
    "American_black_bear": "bear",
    "brown_bear": "bear",
    "ground_beetle": "beetle",
    "leaf_beetle": "beetle",
    "rhinoceros_beetle": "beetle",
    "dung_beetle": "beetle",
    "wild_boar": "boar",
    "ringlet": "butterfly",
    "monarch": "butterfly",
    "sulphur_butterfly": "butterfly",
    "lycaenid": "butterfly",
    "Egyptian_cat": "cat",
    "tabby": "cat",
    "Siamese_cat": "cat",
    "Persian_cat": "cat",
    "lynx": "cat",
    "water_buffalo": "cow",
    "Dungeness_crab": "crab",
    "red_deer": "deer",
    "elk": "deer",
    "Labrador_retriever": "dog",
    "basset": "dog",
    "Border_collie": "dog",
    "Chihuahua": "dog",
    "Bouvier_des_Flandres": "dog",
    "Brittany_spaniel": "dog",
    "English_setter": "dog",
    "Greater_Swiss_Mountain_dog": "dog",
    "Ibizan_hound": "dog",
    "Mexican_hairless": "dog",
    "tox_terrier": "dog",
    "Pekinese": "dog",
    "Pomeranian": "dog", 
    "golden_retriever": "dog",
    "pug": "dog",
    "ass": "donkey",
    "mallard": "duck",
    "bald_eagle": "eagle",
    "golden_eagle": "eagle",
    "African_elephant": "elephant",
    "Indian_elephant": "elephant",
    "Arctic_fox": "fox",
    "red_fox": "fox",
    "grey_fox": "fox",
    "kit_fox": "fox",
    "ibex": "goat",
    "mountain_goat": "goat",
    "Arabian_horse": "horse",
    "Appaloosa": "horse",
    "wallaby": "kangaroo",
    "agama": "lizard",
    "alligator_lizard": "lizard",
    "banded_gecko": "lizard",
    "Komodo_dragon": "lizard",
    "whiptail": "lizard", 
    "American_lobster": "lobster",
    "spiny_lobster": "lobster",
    "house_mouse": "mouse",
    "fiddler_crab": "crab",
    "rock_crab": "crab",
    "king_crab": "crab",
    "magpie": "crow",
    "jay": "crow",
    "drake": "duck",
    "tusker": "elephant",
    "great_grey_owl": "owl",
    "giant_panda": "panda",
    "African_grey": "parrot",
    "macaw": "parrot",
    "sulphur-crested_cockatoo": "parrot",
    "pelican": "pelecaniformes",
    "black_stork": "pelecaniformes",
    "king_penguin": "penguin",
    "hog": "pig",
    "red-backed_sandpiper": "sandpiper",
    "redshank": "sandpiper",
    "dowitcher": "sandpiper",
    "great_white_shark": "shark",
    "hammerhead": "shark",
    "tiger_shark": "shark",
    "horned_viper": "snake",
    "ram": "sheep",
    "bighorn": "sheep",
    "vine_snake": "snake",
    "king_snake": "snake",
    "night_snake": "snake",
    "Indian_cobra": "snake",
    "ringneck_snake": "snake",
    "rock_python": "snake",
    "thunder_snake": "snake",
    "fox_squirrel": "squirrel",
    "black_swan": "swan",
    "box_turtle": "turtle",
    "loggerhead": "turtle",
    "leatherback_turtle": "turtle",
    "mud_turtle": "turtle",
    "grey_whale": "whale",
    "killer_whale": "whale",
    "timber_wolf": "wolf",
    "white_wolf": "wolf"
}

species_recognition_result["y_pred_mapped"] = species_recognition_result["y_pred"].map(
    lambda l: imagenet_to_kaggle.get(l, l) # safe map accessor (labels that are not present in the dict keys, remain unchanged)
)

# Evaluation
Carl et al. provide two kinds of performance metrics: overall model accuracy and the accuracy for each species. By grouping the samples by the true species, it is straightforward to calculate metrics that can be directly compared to the results from Carl et al.

In [37]:
accuracy_score(species_recognition_result["y_true"], species_recognition_result["y_pred_mapped"])

0.62

In [38]:
species_recognition_result.assign(correct = species_recognition_result["y_true"] == species_recognition_result["y_pred_mapped"]).groupby("y_true")["correct"].mean().sort_values(ascending=False)

y_true
bison         1.0
bear          1.0
boar          1.0
crab          1.0
elephant      1.0
             ... 
reindeer      0.0
squid         0.0
sparrow       0.0
turkey        0.0
woodpecker    0.0
Name: correct, Length: 90, dtype: float64

# Summary

# Future Work

# Appendix

## Dependency Table

In [15]:
package_versions = {}
with open("requirements.txt", "r") as f:
    for line in f:
        line = line.strip()
        if not line or line.startswith("#"):
            continue
        if "==" in line:
            pkg, ver = line.split("==")
            package_versions[pkg] = ver
        else:
            package_versions[line] = "unknown"
            
with open("figure/requirements.md", "w") as f:
    f.write(
        pd.DataFrame({
            "package": package_versions.keys(),
            "version": package_versions.values()
        }).to_markdown(index=False)
    )

```{table} 
:name: table-requirements
:align: center

Runtime dependencies
```{include} figure/requirements.md

## Imagenet Label Mapping Table

In [16]:
with open("figure/imagenet_label_mapping.md", "w") as f:
    f.write(
        pd.DataFrame({
            "imagenet label": imagenet_to_kaggle.keys(),
            "mapped label": imagenet_to_kaggle.values()
        }).groupby("mapped label")["imagenet label"].apply(
            lambda labels: ", ".join(labels)
        ).reset_index().to_markdown(index=False)
    )

```{table} 
:name: table-imagenet-label-mapping
:align: center

Imagenet label mapping
```{include} figure/imagenet_label_mapping.md

## Inference Output Table

In [17]:
github_animal_image_paths = [
    f"[{p.parent.name}](https://github.com/tobsel7/research-vetmedwien-animal-species-identification/raw/main/{p})"
    for p in wildlife_image_paths
]

with open("figure/species_recognition_result.md", "w") as f:
    f.write(
        pd.DataFrame({
            "truth": github_animal_image_paths,
            "mapped prediction": species_recognition_result["y_pred_mapped"],
            "model prediction": species_recognition_result["y_pred"]
        }).to_markdown(index=False)
    )

```{table} 
:name: species-recognition-result
:align: center

Inception-ResNet-v2 predictions
```{include} figure/species_recognition_result.md